# Modelo de Regresión Lineal para predecir el precio del aguacate

La **Regresión Lineal** busca modelar la relación lineal entre las variables predictoras (`X`) y la variable objetivo (`y = AveragePrice`).  

En este notebook se realizarán los siguientes pasos:

1. Cargar el dataset limpio y estandarizado.
2. Separar la variable objetivo (`AveragePrice`) de las variables predictoras.
3. Dividir los datos en conjuntos de entrenamiento y prueba.
4. Entrenar un modelo de Regresión Lineal.
5. Evaluar el desempeño del modelo usando MAE, RMSE y R².
6. Exportar los coeficientes del modelo para analizar la importancia de las variables.


In [ ]:

# Regresión Lineal: Preparación y Entrenamiento

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ------------------------------------------------------------
# Cargar dataset final limpio y escalado
# ------------------------------------------------------------
df = pd.read_csv("avocado_final_clean.csv")

# ------------------------------------------------------------
# Separar variable objetivo y predictoras
# ------------------------------------------------------------
y = df['AveragePrice']
X = df.drop(columns=['AveragePrice'])

# ------------------------------------------------------------
# Escalar únicamente las variables numéricas continuas
# ------------------------------------------------------------
# Identificar columnas binarias (0/1)
binary_cols = [col for col in X.columns if set(X[col].unique()).issubset({0, 1})]
# Columnas numéricas continuas
numeric_cols = [col for col in X.columns if col not in binary_cols]

scaler = StandardScaler()
X_scaled_numeric = pd.DataFrame(scaler.fit_transform(X[numeric_cols]), columns=numeric_cols, index=X.index)

# Concatenar con las binarias sin escalar
X_final = pd.concat([X_scaled_numeric, X[binary_cols]], axis=1)

# ------------------------------------------------------------
# Dividir dataset en entrenamiento y prueba
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

# ------------------------------------------------------------
# Entrenar modelo de Regresión Lineal
# ------------------------------------------------------------
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# ------------------------------------------------------------
# Realizar predicciones
# ------------------------------------------------------------
y_pred = lr_model.predict(X_test)

# ------------------------------------------------------------
# Evaluar modelo
# ------------------------------------------------------------
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # raíz cuadrada manual
r2 = r2_score(y_test, y_pred)

print("Resultados del modelo de Regresión Lineal:")
print(f"MAE  (Error Absoluto Medio): {mae:.4f}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse:.4f}")
print(f"R²   (Coeficiente de Determinación): {r2:.4f}")

# ------------------------------------------------------------
#  Importancia de las variables (coeficientes)
# ------------------------------------------------------------
coef_df = pd.DataFrame({
    'Variable': X_final.columns,
    'Coeficiente': lr_model.coef_
}).sort_values(by='Coeficiente', key=abs, ascending=False)

print("Coeficientes de la Regresión Lineal:")
print(coef_df.head(20))


📈 Resultados del modelo de Regresión Lineal:
MAE  (Error Absoluto Medio): 0.1884
RMSE (Raíz del Error Cuadrático Medio): 0.2471
R²   (Coeficiente de Determinación): 0.6088

📌 Coeficientes de la Regresión Lineal:
                      Variable  Coeficiente
31              region_Houston    -0.353755
24        region_DallasFtWorth    -0.352894
41              region_NewYork     0.331903
39            region_Nashville    -0.322252
52              region_Roanoke    -0.306334
58         region_SouthCentral    -0.297138
23             region_Columbus    -0.291354
22     region_CincinnatiDayton    -0.291123
36           region_Louisville    -0.272860
30  region_HartfordSpringfield     0.262911
51      region_RichmondNorfolk    -0.244619
55         region_SanFrancisco     0.243906
26              region_Detroit    -0.239838
40     region_NewOrleansMobile    -0.234268
42            region_Northeast     0.230967
32         region_Indianapolis    -0.222558
35           region_LosAngeles    -0.213